In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
print(tf.__version__)

2.15.0


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
datagen = ImageDataGenerator(rescale=1/255,validation_split=0.2)
train_data=datagen.flow_from_directory('/content/drive/Shareddrives/Garbage_detection',
                                             batch_size=128,
                                             target_size=(300,300),
                                             subset='training',
                                             class_mode='binary')
validation_=datagen.flow_from_directory('/content/drive/Shareddrives/Garbage_detection',
                                                       batch_size=128,
                                                       target_size=(300,300),
                                                       subset='validation',
                                                       class_mode='binary')



Found 2246 images belonging to 3 classes.
Found 560 images belonging to 3 classes.


In [4]:
#plt.figure(figsize=(8,8))
#for images,labels in train_data.take(1):
#  for x in range (9):
#    ax=plt.subplot(3,3,x+1)
#    plt.imshow(images[x].numpy().astype('uint8'))
#    plt.title(class_names[labels[x]])
#    plt.axis('off')

In [5]:
model = Sequential([
    layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 149, 149, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 73, 73, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 35, 35, 64)        0

In [7]:
model.compile(optimizer=RMSprop(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics = ['accuracy'])

In [8]:
#my_callbacks = [
#   tf.keras.callbacks.EarlyStopping(patience=2),
#   tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
#    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
#]

In [9]:
history = model.fit(train_data,
                    epochs=15,
                    verbose=2,
                    validation_data=validation_)

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

In [ ]:
uploaded=files.upload()
for fn in uploaded.keys():
  path='/content/' + fn
  img=load_img(path, target_size=(300, 300))
  x=img_to_array(img)
  x /= 255
  x=np.expand_dims(x, axis=0)
  image_ = np.vstack([x])
  classes = model.predict(image_, batch_size=10)
  print(classes[0])

  if classes[0]>0.5:print(fn + " is garbage")
  else:print(fn + " is not a garbage")

In [ ]:
model.save("mymodel.keras")